In [ ]:
import json

In [ ]:
# Load the original JSON file
with open('/content/GPT4_dev_graph_lmm.json', 'r') as file:
    data = json.load(file)

# Iterate over each instance and remove the "Locations" field
for instance in data:
    if 'Locations' in instance:
        instance.pop('Locations')

# Save the modified data to a new JSON file
with open('/content/GPT4_dev_cmap_llm.json', 'w') as file:
    json.dump(data, file, indent=4)

print("The 'Locations' field has been removed from each instance and the new JSON file has been saved.")


The 'Locations' field has been removed from each instance and the new JSON file has been saved.


In [ ]:

# Load the original JSON file
with open('/content/GPT4_dev_graph_lmm.json', 'r') as file:
    data = json.load(file)

# Create a set to hold unique Instance_id values
unique_instance_ids = set()

# Iterate over each instance and add the Instance_id to the set
for instance in data:
    if 'Instance_id' in instance:
        unique_instance_ids.add(instance['Instance_id'])

# Count the unique instances
unique_count = len(unique_instance_ids)

print(f"The number of unique instances is: {unique_count}")


The number of unique instances is: 95


In [ ]:
import json
from geopy.distance import geodesic

# Load the JSON file
file_path = '/content/dev_visual_environment (1).json'
with open(file_path, 'r') as file:
    data = json.load(file)

def calculate_intermediate_points(location1, location2, num_points):
    """
    Calculate intermediate points between two locations.

    Args:
        location1 (tuple): Coordinates (lat, lon) of the first point.
        location2 (tuple): Coordinates (lat, lon) of the second point.
        num_points (int): Number of intermediate points to generate.

    Returns:
        list: A list of intermediate points [(lat, lon), ...].
    """
    lat1, lon1 = location1
    lat2, lon2 = location2
    intermediate_points = []
    for i in range(1, num_points + 1):
        fraction = i / (num_points + 1)
        lat = lat1 + fraction * (lat2 - lat1)
        lon = lon1 + fraction * (lon2 - lon1)
        intermediate_points.append((lat, lon))
    return intermediate_points

# Process each instance
updated_data = []
for instance in data:
    instance_id = instance["Instance_id"]
    locations = instance["Locations"]
    updated_locations = []

    for i in range(len(locations) - 1):
        loc1 = locations[i]
        loc2 = locations[i + 1]

        # Calculate the distance between consecutive points
        coord1 = (loc1["lat"], loc1["lon"])
        coord2 = (loc2["lat"], loc2["lon"])
        distance = geodesic(coord1, coord2).meters

        # Add the current location
        updated_locations.append(loc1)

        # Determine the number of intermediate points based on distance
        if 100 <= distance < 150:
            num_points = 1
        elif 150 <= distance < 200:
            num_points = 2
        elif distance >= 200:
            num_points = 3
        else:
            num_points = 0

        # Generate and add intermediate points
        if num_points > 0:
            intermediate_points = calculate_intermediate_points(coord1, coord2, num_points)
            heading = loc1["heading"]
            for idx, (lat, lon) in enumerate(intermediate_points, start=1):
                intermediate_location = {
                    "lat": round(lat,7),
                    "lon": round(lon,7),
                    "heading": heading,  # Use the heading from the first point
                    "file_name": f"{instance_id}_intermediate_{i}_{idx}.jpg"
                }
                updated_locations.append(intermediate_location)

    # Add the last location of the instance
    updated_locations.append(locations[-1])
    updated_data.append({
        "Instance_id": instance_id,
        "Locations": updated_locations
    })

# Save the updated data
output_path = '/content/dev_visual_environment.json'
with open(output_path, 'w') as output_file:
    json.dump(updated_data, output_file, indent=4)

output_path



'/content/dev_visual_environment.json'

In [ ]:
# Remove "file_name" from all locations in the updated data
for instance in updated_data:
    for location in instance["Locations"]:
        location.pop("file_name", None)  # Remove the "file_name" key if it exists

# Save the updated data without "file_name"
output_path_no_filename = '/content/dev_visual_environment.json'
with open(output_path_no_filename, 'w') as output_file:
    json.dump(updated_data, output_file, indent=4)

output_path_no_filename


'/content/dev_visual_environment.json'

In [ ]:
# Load the JSON file
file_path = '/content/dev_visual_environment.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Function to calculate directions based on headings
def calculate_directions(routes):
    results = []
    for route in routes:
        instance_id = route["Instance_id"]
        locations = route["Locations"]

        if len(locations) < 2:
            continue  # Skip if fewer than 2 locations

        directions = ["Forward"]  # Start every route with "Forward"

        for i in range(1, len(locations)):
            current_heading = locations[i - 1]["heading"]
            next_heading = locations[i]["heading"]

            # Calculate heading difference
            diff = (next_heading - current_heading) % 360
            if diff > 180:
                diff -= 360  # Normalize to -180 to 180 range

            # Determine direction
            if diff > 15:  # Right turn
                directions.append("Right")
            elif diff < -15:  # Left turn
                directions.append("Left")
            else:  # Minor change treated as Forward
                directions.append("Forward")

        # Remove the last direction from the list
        if directions:
            directions.pop()  # Remove the last item in the list

        # Append the result for this instance
        results.append({
            "Instance_id": instance_id,
            "Directions": directions
        })

    return results

# Process directions from the data
routes = data  # Assuming the top-level JSON contains the routes
directions_output = calculate_directions(routes)

# Save results to a new JSON file
output_path = '/content/dev_visual_environment_directions.json'
with open(output_path, 'w') as output_file:
    json.dump(directions_output, output_file, indent=4)

output_path

'/content/dev_visual_environment_directions.json'